### Install Libraries & Dependency

In [2]:
!pip install --upgrade pip

In [3]:
!pip install protobuf==3.20.3
!pip install absl-py rouge_score nltk
!pip install numpy

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=d79a2f7b816c84aa495a6b019000a40abc195b855561d6a8c24d503505eafae9
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [4]:
!pip install transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 49.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.2
    Uninstalling transformers-4.29.2:
      Successfully uninstalled transformers-4.29.2


In [5]:
!pip install protobuf==3.20.3

In [6]:
!pip install absl-py

In [7]:
!pip install rouge_score 

In [8]:
!pip install rouge

In [9]:
!pip install nltk

In [10]:
!pip install sentencepiece

In [11]:
!pip install datasets

In [12]:
!pip install numpy

In [13]:
!pip install evaluate 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.9 MB/s eta 0:00:00


In [14]:
!pip install --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 5.8 MB/s eta 0:00:0000:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.12.0
    Uninstalling accelerate-0.12.0:
      Successfully uninstalled accelerate-0.12.0


Check if GPU is available and make it enable

In [15]:
import torch
import pandas as pd

if torch.cuda.is_available():
    print("GPU is enabled.")
    print("device count: {}, current device: {}".format(torch.cuda.device_count(), torch.cuda.current_device()))
else:
    print("GPU is not enabled.")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GPU is enabled.
device count: 1, current device: 0


### Data Preparation

Using created dataset for maroon team

In [16]:
from datasets import load_dataset

ds = load_dataset("bgspaditya/maroon100k")
ds

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/bgspaditya--maroon100k-f8cb1371f4070783/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'summary'],
        num_rows: 80000
    })
    test: Dataset({
        features: ['article', 'summary'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['article', 'summary'],
        num_rows: 10000
    })
})

checking data 

In [17]:
ds["train"][0]

{'article': 'Selama musim mudik dan arus balik Idul Fitri 2017, truk tertentu dilarang melintas di ruas jalan tertentu. Termasuk masuk ke jalan tol melalui gerbang di Cikarang, Bekasi, Jawa Barat. Namun, pada akhir pekan ini kendaraan besar yang membawa barang diperbolehkan kembali masuk ke gerbang tol Cikarang. " Truk baru boleh masuk tol Cikarang hari Minggu ( 2 / 6 ), " kata Kapolda Metro Jaya, Irjen M. Iriawan, saat meninjau arus balik di Cikarang, Jumat ( 30 / 6 ). Iriawan menegaskan, selama masa larangan diberlakukan, truk yang coba-coba melintas di jalan tol pasti diberhentikan. " Pasti diberhentikan sebelum masuk, cuma dikoordinasikan dulu sama ( Dinas ) Perhubungan Darat, " sebutnya. Sebagai informasi, mulai H - 4 Idul Fitri pemerintah memberlakukan larangan bagi truk tertentu untuk melalui jalur yang dilalui pemudik. Larangan tersebut tidak berlaku untuk truk pembawa bahan makanan. Truk sembako yang boleh melintas saat Lebaran juga dibatasi, yaitu hanya truk dengan maksimal d

### Preprocessing

Using Tokenizer from multilingualT5-small

In [18]:
from transformers import AutoTokenizer

t5_tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


#### Tokenize data

In [19]:
def tokenize_sample_data(data):
    # Max token size is 14536 and 215 for inputs and labels, respectively.
    # Here I restrict these token size.
    input_feature = t5_tokenizer(data["article"], truncation=True, max_length=1024)
    label = t5_tokenizer(data["summary"], truncation=True, max_length=128)
    return {
        "input_ids": input_feature["input_ids"],
        "attention_mask": input_feature["attention_mask"],
        "labels": label["input_ids"],
    }


In [20]:
tokenized_ds = ds.map(
    tokenize_sample_data,
    remove_columns=["summary", "article"],
    batched=True,
    batch_size=128)
tokenized_ds

  0%|          | 0/625 [00:00<?, ?ba/s]

  0%|          | 0/79 [00:00<?, ?ba/s]

  0%|          | 0/79 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 80000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
})

#### Load Model

In [21]:
from transformers import AutoConfig, AutoModelForSeq2SeqLM

mt5_config = AutoConfig.from_pretrained(
    "google/mt5-small",
    max_length=128,
    length_penalty=0.6,
    no_repeat_ngram_size=2,
    num_beams=15,
)
model = (AutoModelForSeq2SeqLM
         .from_pretrained("google/mt5-small", config=mt5_config)
         .to(device))

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [22]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    t5_tokenizer,
    model=model,
    return_tensors="pt")

#### Define Evaluation Function

In [23]:
!pip install mxnet-mkl==1.6.0 numpy==1.23.1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 MB 14.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 50.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 

In [24]:
!pip install numpy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [25]:
import evaluate
import numpy as np
from nltk.tokenize import RegexpTokenizer

rouge_metric = evaluate.load("rouge")

def tokenize_sentence(arg):
    encoded_arg = t5_tokenizer(arg)
    return t5_tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

def metrics_func(eval_arg):
    preds, labels = eval_arg
    # Replace -100
    labels = np.where(labels != -100, labels, t5_tokenizer.pad_token_id)
    # Convert id tokens to text
    text_preds = t5_tokenizer.batch_decode(preds, skip_special_tokens=True)
    text_labels = t5_tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Insert a line break (\n) in each sentence for ROUGE scoring
    # (Note : Please change this code, when you perform on other languages except for Japanese)
    text_preds = [(p if p.endswith(("!", "！", "?", "？", "。")) else p + "。") for p in text_preds]
    text_labels = [(l if l.endswith(("!", "！", "?", "？", "。")) else l + "。") for l in text_labels]
    sent_tokenizer_jp = RegexpTokenizer(u'[^!！?？。]*[!！?？。]')
    text_preds = ["\n".join(np.char.strip(sent_tokenizer_jp.tokenize(p))) for p in text_preds]
    text_labels = ["\n".join(np.char.strip(sent_tokenizer_jp.tokenize(l))) for l in text_labels]
    # compute ROUGE score with custom tokenization
    return rouge_metric.compute(
        predictions=text_preds,
        references=text_labels,
        tokenizer=tokenize_sentence
    )

Test evaluation on sample data

In [26]:
from torch.utils.data import DataLoader

sample_dataloader = DataLoader(
    tokenized_ds["test"].with_format("torch"),
    collate_fn=data_collator,
    batch_size=5)
for batch in sample_dataloader:
    with torch.no_grad():
        preds = model.generate(
            batch["input_ids"].to(device),
            num_beams=15,
            num_return_sequences=1,
            no_repeat_ngram_size=1,
            remove_invalid_values=True,
            max_length=128,
        )
    labels = batch["labels"]
    break

metrics_func([preds, labels])

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


{'rouge1': 0.1272642390289449,
 'rouge2': 0.09172870140612076,
 'rougeL': 0.13189972154053362,
 'rougeLsum': 0.1284722738652983}

### Modeling

#### Tuning

In [27]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir = "/kaggle/working/mt5-summarize-id",
    log_level = "error",
    num_train_epochs = 3,
    learning_rate = 5e-4,
    lr_scheduler_type = "linear",
    warmup_steps = 90,
    optim = "adafactor",
    weight_decay = 0.01,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 1,
    gradient_accumulation_steps = 16,
    evaluation_strategy = "steps",
    eval_steps = 100,
    predict_with_generate=True,
    generation_max_length = 128,
    save_steps = 500,
    logging_steps = 10,
    push_to_hub = False
)

In [28]:
from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    data_collator = data_collator,
    compute_metrics = metrics_func,
    train_dataset = tokenized_ds["train"],
    eval_dataset = tokenized_ds["validation"].select(range(20)),
    tokenizer = t5_tokenizer,
)

In [29]:
pip install wandb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


#### Training

In [30]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
100,1.876800,1.232928,0.443292,0.373452,0.421335,0.420848
200,1.329700,1.019115,0.473258,0.353598,0.422021,0.421181
300,1.093200,0.938543,0.504651,0.390037,0.433003,0.435729
400,1.182500,1.002589,0.526954,0.399803,0.455854,0.456933
500,1.037200,0.904115,0.531573,0.418176,0.475534,0.473681
600,1.010200,0.850385,0.556471,0.423550,0.481393,0.481885
700,1.003500,0.872639,0.538726,0.404551,0.462357,0.458232
800,0.936800,0.900755,0.602218,0.484716,0.537433,0.536815
900,0.949900,0.850713,0.588445,0.478771,0.533418,0.532150
1000,0.970100,0.852490,0.554236,0.426112,0.484939,0.483797


TrainOutput(global_step=7500, training_loss=0.8749470450719198, metrics={'train_runtime': 18560.6019, 'train_samples_per_second': 12.931, 'train_steps_per_second': 0.404, 'total_flos': 1.4573985355180032e+17, 'train_loss': 0.8749470450719198, 'epoch': 3.0})

#### Saving Fine-Tuned Model

In [31]:
# from google.colab import drive
# drive.mount('/content/drive')

In [32]:
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# /kaggle/working/

In [33]:
import os

os.makedirs("/kaggle/working/trained_for_summarization_id", exist_ok=True)
if hasattr(trainer.model, "module"):
    trainer.model.module.save_pretrained("/kaggle/working/trained_for_summarization_id")
else:
    trainer.model.save_pretrained("/kaggle/working/trained_for_summarization_id")

In [34]:
# import os

# os.makedirs("/content/drive/MyDrive/trained_for_summarization_id", exist_ok=True)
# if hasattr(trainer.model, "module"):
#     trainer.model.module.save_pretrained("/content/drive/MyDrive/trained_for_summarization_id")
# else:
#     trainer.model.save_pretrained("/content/drive/MyDrive/trained_for_summarization_id")

### Using Fine-Tuned Model

Load Model

In [35]:
from transformers import AutoModelForSeq2SeqLM

model = (AutoModelForSeq2SeqLM
         .from_pretrained("/kaggle/working/trained_for_summarization_id")
         .to(device))

In [36]:
model

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

In [37]:
!zip -r model.zip '/kaggle/working/trained_for_summarization_id'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  adding: kaggle/working/trained_for_summarization_id/ (stored 0%)
  adding: kaggle/working/trained_for_summarization_id/config.json (deflated 48%)
  adding: kaggle/working/trained_for_summarization_id/pytorch_model.bin (deflated 21%)
  adding: kaggle/working/trained_for_summarization_id/generation_config.json (deflated 33%)


In [38]:
from IPython.display import FileLink
FileLink(r'model.zip')

/kaggle/working/model.zip

#### Testing Model

Test on sample data

In [39]:
from torch.utils.data import DataLoader
sample_dataloader = DataLoader(
    tokenized_ds["test"].with_format("torch"),
    collate_fn=data_collator,
    batch_size=5)
for batch in sample_dataloader:
    with torch.no_grad():
        preds = model.generate(
            batch["input_ids"].to(device),
            num_beams=15,
            num_return_sequences=1,
            no_repeat_ngram_size=1,
            remove_invalid_values=True,
            max_length=128,
        )
    labels = batch["labels"]
    break
labels = np.where(labels != -100, labels, t5_tokenizer.pad_token_id)
text_preds = t5_tokenizer.batch_decode(preds, skip_special_tokens=True)
text_labels = t5_tokenizer.batch_decode(labels, skip_special_tokens=True)
# Show result
print("***** Input's Text *****")
print(ds["test"]["article"][0])
print("***** Summary Text (True Value) *****")
print(text_labels[0])
print("***** Summary Text (Generated Text) *****")
print(text_preds[0])

***** Input's Text *****
Rimanews - Gempa berkekuatan 6,1 skala Richter mengguncang wilayah dekat kota suci Syiah Mashhad, Iran, hari ini, menurut laporan media setempat. Satu orang tewas akibat gempa yang terjadi di daerah Sepid Sang, sekitar 80 kilometer dari Mashhad, pada pukul 10.39 waktu setempat tersebut. " Mengerikan. Menimbulkan banyak suara bising. Semuanya bergoyang, " kata seorang warga Mashhad kepada AFP melalui telepon. Menurut Survei Geologi Amerika Serikat ( USGS ), gempa, yang juga melukai sejumlah orang dan menyebabkan kerusakan cukup parah tersebut, terjadi pada kedalaman 13 kilometer. Gubernur setempat mengatakan di stasiun televisi pemerintah bahwa gempa mengguncang wilayah yang luas. Juru bicara layanan darurat Mojtaba Khaledi mengatakan, jumlah korban di desa-desa dan kota kecil mungkin besar, seperti dikutip kantor berita pemerintah IRNA. Tim Bulan Sabit Merah dan layanan darurat sudah dikerahkan ke area bencana, tambahnya. Seorang pejabat Bulan Sabit Merah menga

In [40]:
print("***** Input's Text *****")
print(ds["test"]["article"][2])
print("***** Summary Text (True Value) *****")
print(text_labels[2])
print("***** Summary Text (Generated Text) *****")
print(text_preds[2])

***** Input's Text *****
Tiga tanker terbakar di Pelabuhan Pintu Air, Merauke, Papua, Rabu  sekitar pukul 10. 00 WIT. Hingga berita ini disusun, kobaran api telah menghanguskan dua kapal pengangkut bahan bakar minyak, yakni kapal Maros Jaya I dan kapal Nurhikmah II. Sedangkan tanker lain, Nusantara Jaya, masih terbakar. Sejumlah anak buah kapal Maros Jaya I menuturkan, saat itu mereka mendengar suara ledakan dari ruang mesin. Api langsung membesar dan menyambar bagian kapal lainnya. Tak lama berselang, api menyambar dua kapal lainnya. Kapal Maros Jaya I dan Nurhikmah ludes terbakar hingga karam. Sedangkan kobaran api saat ini membakar buritan kapal Nusantara Jaya. Ketiga kapal itu rata-rata mengangkut 70 ton BBM. Kejadian ini bukanlah yang pertama di Pelabuhan Pintu Air, Merauke. Desember tahun silam, peristiwa serupa menimpa dua tanker. Atas kejadian yang berulang, kalangan pengusaha kapal di sana menyesalkan ketidaktegasan administratur pelabuhan atau adpel setempat. Sebab, pihak adp

Test on An Example Article

In [41]:
ARTICLE_TO_SUMMARIZE = "Still Life, Vase with Daisies, and Poppies dibuat beberapa bulan sebelum van Gogh meninggal. Still Life, Vase with Daisies, and Poppies melampaui perkiraan sebelumnya US$50 juta pada pelelangan di Sotheby. Lukisan Van Gogh dibuat di rumah dokter Prancisnya hanya beberapa bulan sebelum dirinya meninggal di tahun 1890. Karyanya ini adalah satu dari beberapa lukisannya yang terjual saat Van Gogh masih hidup. Rekor penjualan pelukis Belanda pada balai lelang adalah US$82,5 juta lewat Portrait of Dr Gachet. Lukisan tentang dokternya ini terjual pada tahun 1990. Pada pelelangan ini patung Alberto Giacometti dari tahun1951 terjual seharga lebih dari US$100 juta, di bawah rekor karya seniman Swiss itu senilai US$104,3 juta. Sementara pematung Amedeo Modigliani mencatat rekor baru lewat Tete patung kepala yang dibuat di tahun 1911-1912 dari batu kapur terjual lebih US$70 juta."
TRUE_SUMM = "Sebuah mahakarya Vincent van Gogh, yang dilukis pada akhir kehidupannya, terjual di New York sebesar US$61,8 juta atau Rp752 miliar."

In [42]:
input_ids = t5_tokenizer.encode(ds["test"]["article"][0], return_tensors='pt', truncation=True, max_length=1024)
label = t5_tokenizer.encode(text_labels[0], truncation=True, return_tensors='pt', max_length=128)

In [43]:
# THIS MODEL
with torch.no_grad():
  preds = model.generate(
              input_ids.to(device),
              num_beams=15,
              num_return_sequences=1,
              no_repeat_ngram_size=1,
              remove_invalid_values=True,
              max_length=128,
          )

In [44]:
text_preds1 = t5_tokenizer.batch_decode(preds, skip_special_tokens=True)
print(text_preds1[0])

Gempa berkekuatan 6,1 skala Richter mengguncang wilayah Iran, hari ini. Satu orang tewas akibat gempa yang terjadi di daerah Sepid Sang dan 80 kilometer dari Mashhad pada pukul 10.39 saat setempat tersebut


### Evaluation

In [45]:
eval_model_owned = metrics_func([preds, label])
eval_model_owned

{'rouge1': 0.5977011494252873,
 'rouge2': 0.5116279069767441,
 'rougeL': 0.5862068965517242,
 'rougeLsum': 0.5862068965517242}

#### Comparing Model

In [46]:
# CAHYA MODEL
summary_ids = model.generate(input_ids.to(device),
            min_length=20,
            max_length=80,
            num_beams=10,
            repetition_penalty=2.5,
            length_penalty=1.0,
            early_stopping=True,
            no_repeat_ngram_size=2,
            use_cache=True,
            do_sample = True,
            temperature = 0.8,
            top_k = 50,
            top_p = 0.95)

In [47]:
summary_text = t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary_text)

Gempa berkekuatan 6,1 skala Richter mengguncang wilayah dekat kota suci Syiah Mashhad, Iran, hari ini, menurut laporan media setempat. Satu orang tewas akibat gempa yang terjadi di daerah Sepid Sang, sekitar 80 kilometer dari Mashhadi, pada pukul 10.39 waktu se tempat tersebut. " Mengerikan.


In [48]:
eval_model_cahya = metrics_func([summary_ids, label])
eval_model_cahya

{'rouge1': 0.7524752475247526,
 'rouge2': 0.7200000000000001,
 'rougeL': 0.7524752475247526,
 'rougeLsum': 0.7524752475247526}

Compare ROUGE Score

In [49]:
evals = [eval_model_owned, eval_model_cahya]
rouge_scores = pd.concat([pd.Series(d) for d in evals], axis=1).fillna(0).T
rouge_scores.index = ['Owned T5 Model', 'Cahya Model']
rouge_scores

,rouge1,rouge2,rougeL,rougeLsum
Owned T5 Model,0.597701,0.511628,0.586207,0.586207
Cahya Model,0.752475,0.720000,0.752475,0.752475


In [50]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [51]:
from rouge import Rouge
ROUGE = Rouge()

In [52]:
ROUGE.get_scores(summary_text, TRUE_SUMM)

[{'rouge-1': {'r': 0.15, 'p': 0.06666666666666667, 'f': 0.09230768804733748},
  'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-l': {'r': 0.1, 'p': 0.044444444444444446, 'f': 0.0615384572781068}}]

In [53]:
ROUGE.get_scores(text_preds1[0], TRUE_SUMM)

[{'rouge-1': {'r': 0.15, 'p': 0.09090909090909091, 'f': 0.1132075424706303},
  'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-l': {'r': 0.1, 'p': 0.06060606060606061, 'f': 0.07547169341402665}}]

In [54]:
# input_ids = t5_tokenizer.encode(ARTICLE_TO_SUMMARIZE, return_tensors='pt', truncation=True, max_length=1024)
# with torch.no_grad():
#   preds = model.generate(
#               input_ids.to(device),
#               num_beams=15,
#               num_return_sequences=1,
#               no_repeat_ngram_size=1,
#               remove_invalid_values=True,
#               max_length=128,
#           )
# text_preds1 = t5_tokenizer.batch_decode(preds, skip_special_tokens=True)
# print(text_preds1[0])